In [1]:
import pandas as pd
import numpy as np
import datetime
from multiprocessing import Pool
import types

#Read file and select relative columns
fire_data = pd.read_csv('FireData.csv')
fire_selected_col = fire_data[['Date','Datetime','Surface Temperature (Celcius)','Confidence']]
ST = fire_data['Surface Temperature (Celcius)']

# Sort

In [2]:
def qsort(arr): 

    """ 
    Quicksort a list
    
    Arguments:
    arr -- the input list to be sorted

    Return:
    result -- the sorted arr
    """
    if len(arr) <= 1:
        return arr
    else:
        return qsort([x for x in arr[1:] if x[1] < arr[0][1]]) + [arr[0]] + qsort([x for x in arr[1:] if x[1] >= arr[0][1]])

# Round-robin Partition (Use)

In [3]:
# Round-robin data partitionining function
def rr_partition(data, n):
    """
    Perform data partitioning on data

    Arguments:
    data -- an input dataset which is a list
    n -- the number of processors

    Return:
    result -- the paritioned subsets of D
    """
    result = []
    for i in range(n):
        result.append([])
    
    ### START CODE HERE ### 
    
    # Calculate the number of the elements to be allocated to each bin
    n_bin = len(data)/n
    
    # For each bin, perform the following
    for index, element in enumerate(data): 
        # Calculate the index of the bin that the current data point will be assigned
        index_bin = (int) (index % n)
        result[index_bin].append((index,element)) #need index
    ### END CODE HERE ###
    
    return result

# Hash Parition (Unuse)

In [4]:
def s_hash(x, n):
    ### START CODE HERE ###
    result = x%n
    ### END CODE HERE ###
    return result

# Hash data partitionining function.
# We will use the "s_hash" function defined above to realise this partitioning
def h_partition(data, n):

### START CODE HERE ###
    dic = {} # We will use a dictionary
    for i, x in enumerate(data): # For each data record, perform the following
        h = s_hash(x, n) # Get the hash key of the input
        if (h in dic.keys()): # If the key exists
            l = dic[h]
            l.append((i,x))
            dic[h] = l # Add the new input to the value set of the key
        else: # If the key does not exist
            l = [] # Create an empty value set
            l.append((i,x))
            dic[h] = l # Add the value set to the key
### END CODE HERE ###
    return dic

# Merge

In [5]:
# Let's first look at 'k-way merging algorithm' that will be used 
# to merge sub-record sets in our external sorting algorithm.
import sys

# Find the smallest record
def find_min(records):    
    """ 
    Find the smallest record
    
    Arguments:
    records -- the input record set

    Return:
    result -- the smallest record's index
    """
    m = records[0]
    index = 0
    for i in range(len(records)):
        if(records[i][1] < m[1]):  
            index = i
            m = records[i]
    return index

def k_way_merge(record_sets):
    """ 
    K-way merging algorithm
    
    Arguments:
    record_sets -- the set of mulitple sorted sub-record sets

    Return:
    result -- the sorted and merged record set
    """
    
    # indexes will keep the indexes of sorted records in the given buffers
    indexes = []
    for x in record_sets:
        indexes.append(0) # initialisation with 0

    # final result will be stored in this variable
    result = []  
    
    # the merging unit (i.e. # of the given buffers)
    tuple = []
    
    while(True):
        tuple = [] # initialise tuple
        
        # This loop gets the current position of every buffer
        for i in range(len(record_sets)):
            if(indexes[i] >= len(record_sets[i])):
                tuple.append((sys.maxsize,sys.maxsize))
            else:
                tuple.append(record_sets[i][indexes[i]])  
        
        # find the smallest record 
        smallest = find_min(tuple)
    
        # if we only have sys.maxsize on the tuple, we reached the end of every record set
        if(tuple[smallest][1] == sys.maxsize):
            break

        # This record is the next on the merged list
        result.append(record_sets[smallest][indexes[smallest]])
        indexes[smallest] +=1
   
    return result

In [6]:
# Test k-way merging method
buffers = [[(8, 33), (4, 54), (0, 68), (12, 75)], 
           [(24, 38), (28, 44), (16, 60), (20, 64)], 
           [(184, 39), (176, 49), (180, 50), (188, 50)]]
result = k_way_merge(buffers)
inds = [x[0] for x in result]

# Parallel Processing

In [7]:
# Include this package for parallel processing
import multiprocessing as mp

def parallel_merge_all_sorting(dataset, n_processor, buffer_size):
    """
    Perform a parallel merge-all sorting method

    Arguments:
    dataset -- entire record set to be sorted
    n_processor -- number of parallel processors
    buffer_size -- buffer size determining the size of each sub-record set
    Return:
    result -- the merged record set
    """
    if (buffer_size <= 2):
        print("Error: buffer size should be greater than 2")
        return
    
    result = []

    ### START CODE HERE ### 
    
    # Pre-requisite: Perform data partitioning using round-robin partitioning
    subsets = rr_partition(dataset, n_processor)
    
    # Pool: a Python method enabling parallel processing. 
    pool = mp.Pool(processes = n_processor)

    # ----- Sort phase -----
    # Implement here
    local_sorted_list = []
    for i in subsets:
        local=pool.apply(qsort,[i])
        local_sorted_list.append(local)
    # ---- Final merge phase ----
    #Implement here
    dataset=local_sorted_list
    merge_buffer_size = buffer_size-1
    while True:
        merged_set = []
        N = len(dataset)
        start_pos = 0
        while True:
            if ((N - start_pos) > merge_buffer_size):
                # read C-record sets from the merged record sets, where C = merge_buffer_size
                subset = dataset[start_pos:start_pos + merge_buffer_size]
                merged_set.append(k_way_merge(subset)) # merge lists in subset
                start_pos += merge_buffer_size
            else:
                # read C-record sets from the merged sets, where C is less than merge_buffer_size
                subset = dataset[start_pos:]
                merged_set.append(k_way_merge(subset)) # merge lists in subset
                break

        dataset = merged_set
        if (len(dataset) <= 1): # if the size of merged record set is 1, then stop 
            result = merged_set
            break    
    
    ### END CODE HERE ###
    
    return result

In [8]:
result = parallel_merge_all_sorting(ST, 4, 4)
inds = [x[0] for x in result[0]]
fire_selected_col.loc[inds]


,Date,Datetime,Surface Temperature (Celcius),Confidence
312,2017-07-02,2017-07-02T04:28:42,28,50
311,2017-07-02,2017-07-02T04:28:42,28,50
101,2017-11-11,2017-11-11T15:08:00,29,51
313,2017-07-01,2017-07-01T13:11:41,29,53
186,2017-10-02,2017-10-02T23:44:31,29,50
314,2017-07-01,2017-07-01T13:11:41,29,53
185,2017-10-03,2017-10-03T01:22:44,31,54
47,2017-11-30,2017-11-30T15:38:32,31,61
316,2017-07-01,2017-07-01T03:46:08,32,61
5,2017-12-24,2017-12-24T13:12:01,32,65


### Maybe need to modify the format of output data. That's all pandas. easy peasy

# Q 4.1

# Groupby

In [9]:
# The first step in the merge-all groupby method
def local_groupby(dataset):
    """
    Perform a local groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record set according to the group_by attribute index
    """
    dict = {}
    for record in dataset:
        if isinstance(record, tuple) or isinstance(record, list):
            key = record[0]
            val = record[1]
            if key not in dict:
                dict[key] = 0
            dict[key] += val
        else:    
            key = record
            if key not in dict:
                dict[key] = 0
            dict[key] += 1
    return dict

In [10]:
# Test local_groupby
data = fire_data[['Date']]
data = [x[0] for x in data.values]
grouped_data = local_groupby(data)

# Partition

In [11]:
# Round-robin data partitionining function
def rr_partition(data, n):
    """
    Perform data partitioning on data

    Arguments:
    data -- an input dataset which is a list
    n -- the number of processors

    Return:
    result -- the paritioned subsets of D
    """
    result = []
    for i in range(n):
        result.append([])
    
    ### START CODE HERE ### 
    
    # Calculate the number of the elements to be allocated to each bin
    n_bin = len(data)/n
    
    # For each bin, perform the following
    for index, element in enumerate(data): 
        # Calculate the index of the bin that the current data point will be assigned
        index_bin = (int) (index % n)
        result[index_bin].append(element) #need index
    ### END CODE HERE ###
    
    return result

In [12]:
# Test partition
data_partition = rr_partition(data,4)
# for x in data_partition:
#     print(local_groupby(x))

# Parallel

In [13]:
import multiprocessing as mp

def parallel_merge_all_groupby(dataset):
    """
    Perform a parallel merge_all groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record dictionary according to the group_by attribute index
    """
    
    result = {}

    ### START CODE HERE ### 
    
    # Define the number of parallel processors: the number of sub-datasets.
    n_processor = len(dataset)
    # Pool: a Python method enabling parallel processing. 
    pool = mp.Pool(processes = n_processor)
    # ----- Local aggregation step -----
    local_result = []
    for s in dataset:
        # call the local aggregation method
        local_result.append(pool.apply(local_groupby, [s]))
    pool.close()
    
    new_set=[]
    for element in local_result:
        new_set.extend(list(set(element.items())))

    # ---- Global aggregation step ----
    # Let's assume that the global operator is sum.
    # Implement here
     
    result=local_groupby(new_set)
    ### END CODE HERE ###
    
    return result

In [14]:
result = parallel_merge_all_groupby(data_partition)

### Modify output format if require

# Q4.2

In [15]:
fire_selected_col = fire_data[['Date','Surface Temperature (Celcius)']]

In [16]:
# The first step in the merge-all groupby method
def local_groupby2(dataset):
    """
    Perform a local groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record set according to the group_by attribute index
    """

    dict = {}
    for record in dataset:
        key = record[0]
        val = record[1]
        if key not in dict:
            dict[key] = [0,0]
        dict[key][0] += 1
        dict[key][1] += val

    return dict

In [17]:
# The first step in the merge-all groupby method
def local_groupby3(dataset):
    """
    Perform a local groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record set according to the group_by attribute index
    """

    dict = {}
    for record in dataset:
        if isinstance(record[1], tuple) or isinstance(record[1], list):
            key = record[0]
            val = record[1]
            if key not in dict:
                dict[key] = [0,0]
            dict[key][0] += val[0]
            dict[key][1] += val[1]
        else:
            key = record[0]
            val = record[1]
            if key not in dict:
                dict[key] = [0,0]
            dict[key][0] += 1
            dict[key][1] += val
    for x in dict.items():
        
        dict[x[0]] = x[1][1]/x[1][0]
    

    return dict

In [31]:
# test local_groupby3
aaa = local_groupby3(fire_selected_col.values)

# Same partiontion method within Q 4.1

In [19]:
#test partition
subset =  rr_partition(fire_selected_col.values,4)

In [24]:
import multiprocessing as mp
def parallel_merge_all_groupby2(dataset):
    """
    Perform a parallel merge_all groupby method

    Arguments:
    dataset -- entire record set to be merged

    Return:
    result -- the aggregated record dictionary according to the group_by attribute index
    """
    
    result = {}

    ### START CODE HERE ### 
    
    # Define the number of parallel processors: the number of sub-datasets.
    n_processor = len(dataset)
    # Pool: a Python method enabling parallel processing. 
    pool = mp.Pool(processes = n_processor)
    # ----- Local aggregation step -----
    local_result = []
    for s in dataset:
        # call the local aggregation method
        local_result.append(pool.apply(local_groupby2, [s]))
    pool.close()
    
    new_set=[]
    for element in local_result:
        new_set.extend(element.items())

    # ---- Global aggregation step ----
    # Let's assume that the global operator is sum.
    # Implement here
    result=local_groupby3(new_set)
    ### END CODE HERE ###
    
    return result

In [30]:
result2 = parallel_merge_all_groupby2(subset)

# Same